In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as tf_keras

- 챗GPT<br>
질문1 -> 질문1 + 다음단어1<br>
질문2 -> 질문2 + (질문1 + 다음단어1) + 다음단어2

In [2]:
with open("Data/nietzsche.txt", "rt") as f:
    data = f.read()

In [3]:
print(len(data))
print(data[:30])
data[:30]

600893
PREFACE


SUPPOSING that Truth


'PREFACE\n\n\nSUPPOSING that Truth'

In [4]:
data_lower = data.lower()
data_lower[:30]

'preface\n\n\nsupposing that truth'

In [5]:
print(np.unique(list(data_lower)))
len(np.unique(list(data_lower)))

['\n' ' ' '!' '"' "'" '(' ')' ',' '-' '.' '0' '1' '2' '3' '4' '5' '6' '7'
 '8' '9' ':' ';' '=' '?' '[' ']' '_' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i'
 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' 'ä'
 'æ' 'é' 'ë']


57

In [6]:
print(set(data_lower))
sorted_chars = sorted( set(data_lower))
print(sorted_chars)
print("*" * 100)
char_to_idx = {ch:i for i, ch in enumerate(sorted_chars)}

idx_to_char = {i:ch for i, ch in enumerate(sorted_chars)}
idx_to_char2 = {i:ch for ch, i in char_to_idx.items()}
print(char_to_idx)
print("*" * 100)
print(idx_to_char)
print("*" * 100)
print(idx_to_char2)

{'c', 't', 'k', '6', 's', 'o', '-', 'v', "'", '=', ';', 'h', 'i', ')', 'n', '[', '!', 'ë', 'm', 'w', '1', 'æ', '2', 'y', 'x', 'b', 'u', '"', 'j', '_', 'g', '(', 'p', ' ', 'e', 'f', 'l', ',', '8', '5', '0', '7', '.', 'z', 'r', 'a', 'd', '3', '9', ']', 'ä', 'q', '\n', '4', 'é', ':', '?'}
['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ä', 'æ', 'é', 'ë']
****************************************************************************************************
{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '=': 22, '?': 23, '[': 24, ']': 25, '_': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 

In [7]:
#학습내용
# n개의 연속된 문자 -> n+1번재 문자 예측

In [8]:
len(data_lower)

600893

In [9]:
sequence_length = 50 #연속된 문자의 수
step = 3 # stride (3개 문자씩 이동하면서 데이터 추출)

sequences = [] # (batch 크기 : 한번에 처리하는 갯수, 입력문자갯수, 단어사전크기)
next_chars = [] #배치크기, 단어사전크기

for idx in range(0, len(data_lower) - sequence_length, step):
    sequences.append(data_lower[idx:idx+sequence_length])
    next_chars.append(data_lower[idx+sequence_length])

print(len(sequences), len(next_chars))
print(sequences[0], next_chars[0])

200281 200281
preface


supposing that truth is a woman--what th e


In [10]:
print(next_chars[0])
print()
sequences[0]

e



'preface\n\n\nsupposing that truth is a woman--what th'

In [11]:
x = np.zeros(shape=(len(sequences), sequence_length, len(sorted_chars)))
y = np.zeros(shape=(len(sequences), len(sorted_chars)))

for si, sequence in enumerate(sequences): # si는 입력 문장 번호
    # print(si, sequence)
    for ci, ch in enumerate(sequence): #ci는 한개의 입력문장 안의 문자 순서 번호
        x[si, ci, char_to_idx[ch]] = 1
        y[si, char_to_idx[next_chars[si]]] = 1
        # print(ci, ch, end=", ")
        pass
    
    # if si == 2:
    #     break

In [12]:
x.shape, y.shape

((200281, 50, 57), (200281, 57))

In [23]:
import tensorflow.keras as tf_keras

# 입력 레이어
input = tf_keras.layers.Input(shape=(sequence_length, len(sorted_chars)))  # (50, 57)

lstm = tf_keras.layers.LSTM(units=32, return_sequences=True)(input)  # 시퀀스를 유지
lstm = tf_keras.layers.LSTM(units=32)(lstm)  # 시퀀스를 유지

# 출력 레이어
output = tf_keras.layers.Dense(units=len(sorted_chars), activation="softmax")(lstm)

# 모델 정의
model = tf_keras.models.Model(inputs=input, outputs=output)

# 모델 요약 출력
model.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 50, 57)]          0         
                                                                 
 lstm_3 (LSTM)               (None, 50, 32)            11520     
                                                                 
 lstm_4 (LSTM)               (None, 32)                8320      
                                                                 
 dense_2 (Dense)             (None, 57)                1881      
                                                                 
Total params: 21721 (84.85 KB)
Trainable params: 21721 (84.85 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
model.compile(loss = "categorical_crossentropy", optimizer=tf_keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])

In [26]:
history = model.fit(x, y, batch_size=256, epochs= 10, verbose=1)

Epoch 1/10
783/783 [==============================] - 48s 56ms/step - loss: 2.4638 - accuracy: 0.2985
Epoch 2/10
783/783 [==============================] - 44s 56ms/step - loss: 2.0796 - accuracy: 0.3899
Epoch 3/10
783/783 [==============================] - 43s 55ms/step - loss: 1.9768 - accuracy: 0.4157
Epoch 4/10
783/783 [==============================] - 44s 56ms/step - loss: 1.9197 - accuracy: 0.4311
Epoch 5/10
783/783 [==============================] - 43s 55ms/step - loss: 1.8814 - accuracy: 0.4421
Epoch 6/10
783/783 [==============================] - 43s 55ms/step - loss: 1.8557 - accuracy: 0.4504
Epoch 7/10
783/783 [==============================] - 46s 58ms/step - loss: 1.8309 - accuracy: 0.4569
Epoch 8/10
783/783 [==============================] - 46s 59ms/step - loss: 1.8116 - accuracy: 0.4625
Epoch 9/10
783/783 [==============================] - 49s 62ms/step - loss: 1.7931 - accuracy: 0.4670
Epoch 10/10
783/783 [==============================] - 43s 55ms/step - loss: 1.779

In [17]:
def select_character(preds, temperature=1.0): # temperature 값이 작을 수록 낮은 확률의 값이 선택 가능성이 낮짐
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)    
    probas = np.random.multinomial(1, preds, 1) # 주어진 확률에 따라 다음 값 랜덤 선택
    return np.argmax(probas)

In [21]:
start_idx = np.random.randint(0, len(data_lower) - sequence_length)
seed_text = data_lower[ start_idx: start_idx + sequence_length]
print(seed_text)
print("*" * 100)

for idx in range(100):
    sample = np.zeros(shape=(1, sequence_length, len(sorted_chars))) # 1, 50, 57
    for cidx, c in enumerate(seed_text):
        sample[0, ci, char_to_idx[c]] = 1

    predicted_values = model.predict(sample, verbose=0)
    #selected_char_idx = predicted_values[0].argmax()
    selected_char_idx = select_character(predicted_values[0], 1)
    full_text = sorted_chars[selected_char_idx]
    seed_text = full_text[-50:]
    #seed_text = full_text[idx+1:]

    print(sorted_chars[selected_char_idx], end="")


es life!

25. after such a cheerful commencement, 
****************************************************************************************************
 poned ones inerenenes fed. ly ouly, maty in is,

es ounes ty hen aged inedinedaranes. ineres. tes--

In [33]:
import numpy as np

print( "np.log([0.7, 0.2, 0.1] =", np.log([0.7, 0.2, 0.1]) )
print( "np.log([0.7, 0.2, 0.1] / 0.5 =", np.log([0.7, 0.2, 0.1]) / 0.5 )
print( "np.log([0.7, 0.2, 0.1] / 100 =", np.log([0.7, 0.2, 0.1]) / 100 )
exp_preds = np.exp( np.log([0.7, 0.2, 0.1]) / 3 )
print( exp_preds )
preds = exp_preds / np.sum(exp_preds)
print(preds)
#probas = np.random.multinomial(1, preds, 1)
probas = np.random.multinomial(10, preds, 1)
print(probas)
print( np.argmax(probas) )

np.log([0.7, 0.2, 0.1] = [-0.35667494 -1.60943791 -2.30258509]
np.log([0.7, 0.2, 0.1] / 0.5 = [-0.71334989 -3.21887582 -4.60517019]
np.log([0.7, 0.2, 0.1] / 100 = [-0.00356675 -0.01609438 -0.02302585]
[0.887904   0.58480355 0.46415888]
[0.45842294 0.30193282 0.23964424]
[[5 4 1]]
0
